<a href="https://colab.research.google.com/github/NDU-CSC413/cuda1/blob/master/cuda1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Google colab creates a Linux instance that can be accesses from a Jupyter notebook. A Jupyter notebook consists of a sequence of cells. For our purpose there are two types of cells: **code** and **text**. For both cases you can "execute" the cell by pressing SHIFT-ENTER. For a text cell it will format it. You can always edit a text cell by double-clicking it.
A code cell executes Python by default. In our case we would like to edit c++ (with CUDA extensions) files, compile and run them.

First make sure that the runtime is GPU. From the menu choose Runtime->change runtime type-> Hardware accelerator choose GPU.

We can run arbitrary shell commands by either preceding each one of them with a "!" or we can use the magic characters "%%bash" in the begining of the cell.

As a first example let us determine the type of GPU that the instance has using the nvidia-smi command

In [ ]:
%%bash
nvidia-smi

Mon Apr  4 08:43:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Example 0: Querying the device

In [1]:
%%writefile device.cu
#include <iostream>


int main()
{
	int device;

	cudaDeviceProp properties;
	cudaError_t err = cudaSuccess;
	err = cudaGetDevice(&device);
	err = cudaGetDeviceProperties(&properties, device);
	std::cout << "processor count" << properties.multiProcessorCount << std::endl;
	std::cout << "warp size " << properties.warpSize << std::endl;
	std::cout << "name=" << properties.name << std::endl;
	std::cout << "Compute capability " << properties.major << "." << properties.minor << "\n";
	std::cout << "shared Memory/SM " << properties.sharedMemPerMultiprocessor
		<< std::endl;

	if (err == cudaSuccess)
		printf("device =%d\n", device);
	else
		printf("error getting deivce\n");
	return 0;
}

Writing device.cu


In [3]:
!nvcc -o device device.cu -run

processor count=40
warp size= 32
name=Tesla T4
Compute capability=7.5
shared Memory/SM=65536
device =0


## Example1

To write code, create a code cell and write %%writefile filename.cu at the beginning. To actually create/modify the file "run" the cell

In [4]:
%%writefile example1.cu
#include <iostream>
__global__ void kernel(){
  printf("Hello from GPU\n");
}
int main(){
    kernel<<<1,2>>>();
    cudaDeviceSynchronize();

    
}

Writing example1.cu


At this point we can open the file from the left pane. Click on the folder icon in the left pane to see all the files.
Alternatively, we can list them using the unix "ls" command.

**IMPORTANT**: any file created from a Jupyter notebook will be lost once the instance is restarted.

A colab instance is a actually a VM running linux so each time it starts "from scratch". If you want to save files, mount your Google drive from the left pane.

In [ ]:
!ls -la

total 20
drwxr-xr-x 1 root root 4096 Apr  4 08:44 .
drwxr-xr-x 1 root root 4096 Apr  4 08:37 ..
drwxr-xr-x 4 root root 4096 Mar 23 14:21 .config
-rw-r--r-- 1 root root  111 Apr  4 08:44 example1.cu
drwxr-xr-x 1 root root 4096 Mar 23 14:22 sample_data


NVIDIA nvcc compiler. The file extension MUST be .cu, otherwise it compiles it with a "regular" c++ compiler.

In [5]:
!nvcc example1.cu -o example1 
!./example1

Hello from GPU
Hello from GPU


## Example2

Most of the examples in the workshop use **managed memory**. In the following examples we will manage the memory manually. We will revisit managed memory later.

Before computing on the GPU we need to transfer the data from host memory to device memory. Once the computation is done we transfer it back to the host.
Below is a simple example of that process.

In [6]:
%%writefile example2.cu
#include <iostream>
__global__ void kernel(int *x,int *y,int *z){
    *z=*x+*y;
}
int main(){
    int a=1,b=2,c=0; //host variables
    int *d_a,*d_b,*d_c;//will hold device addresses
    // allocate memory for one integer and store the
     // address in d_a 
    cudaMalloc(&d_a,sizeof(int));
    cudaMalloc(&d_b,sizeof(int));
    cudaMalloc(&d_c,sizeof(int));
    // copy the value of a and b
    // TO device FROM host
    cudaMemcpy(d_a,&a,sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(d_b,&b,sizeof(int),cudaMemcpyHostToDevice);
    kernel<<<1,1>>>(d_a,d_b,d_c);
    // copy the result TO host FROM device
    cudaMemcpy(&c,d_c,sizeof(int),cudaMemcpyDeviceToHost);
    cudaDeviceSynchronize();
    
std::cout<<"value of c is "<<c<<"\n";
    
}

Writing example2.cu


In [7]:
!nvcc example2.cu -o example2
!./example2

value of c is 3


## Example3
This is the first example where we use parallelism, computing the sum of two arrays.
The computation is performed where each thread computes the sum of two elements. To accomplish that we map the thread id to the array index. In this example we use a __single__, __linear__, block therefore the thread id is equal to the builtin variable threadIdx.x

In [ ]:
%%writefile example3.cu
#include <iostream>

__global__ void kernel(float* a, float* b, float* c) {
	int id = threadIdx.x;
	c[id] = a[id] + b[id];
}

int main() {
	int N = 1024;
	float* a, * b, * c;
	float* da, * db, * dc;
  /* allocate memory on host */
	a = (float*)malloc(N * sizeof(float));
	b = (float*)malloc(N * sizeof(float));
	c = (float*)malloc(N * sizeof(float));
  /* allocate memory on device */
	cudaMalloc(&da, N * sizeof(float));
	cudaMalloc(&db, N * sizeof(float));
	cudaMalloc(&dc, N * sizeof(float));
  /* initialize the arrays a and b */
	for (int i = 0; i < N; ++i) {
		a[i] = i;
		b[i] = 2 * i;
	}
  /* copy arrays a and b to device */
	cudaMemcpy(da, a, N * sizeof(float), cudaMemcpyHostToDevice);
	cudaMemcpy(db, b, N * sizeof(float), cudaMemcpyHostToDevice);
/* launch kernel with one block of N threads */
	kernel << <1, N >> > (da, db, dc);
  /* copy result to host */
	cudaMemcpy(c, dc, N * sizeof(float), cudaMemcpyDeviceToHost);
  /* print the first 10 elements */
	for (int i = 0; i < 10; ++i)
		std::cout << c[i] << ' ';
	std::cout << std::endl;
	/* free memory on host and device */
	free(a);
	free(b);
	free(c);
	cudaFree(db);
	cudaFree(dc);
	cudaFree(da);

}

Overwriting example3.cu


In [ ]:
!nvcc example3.cu -o example3
!./example3

0 3 6 9 12 15 18 21 24 27 


## Thread blocks

In CUDA the __maximum__ number of threads in a block is 1024. What if in the previous example we would like to compute the sum of two vectors with size bigger than 1024? We use multiple blocks.

### Example4
We repeat the previous example by using multiple blocks.